# Analysis of NYC Taxi Cab Data
## Import libraries and connect to database

In [37]:
import pandas as pd
import numpy as np
import sqlite3
import os
import datetime as dt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# set working directory up a level from this file
os.chdir('..')

# Connect to database:
con = sqlite3.connect("NYC-Taxi.db")

## Extract observations and clean data

The analysis will focus on sample of 1,000,000 observations of green and yellow cab data. Data from each of these sources will be stacked together, features created, and then finally a train and test set created.

In [38]:
# need to set seed by randomly sampling indices in python then pass to sql

# Store sample of green and yellow cab data into dataframes:
df1 = pd.read_sql(f"SELECT * FROM green_cabs ORDER BY random() LIMIT 500000", con=con)
df2 = pd.read_sql(f"SELECT * from yellow_cabs ORDER BY random() LIMIT 500000", con=con)

# Add labels for green and yellow cabs and rename pickup/dropoff datetime columns:
df1 = df1.rename(columns={"lpep_pickup_datetime": "pickup_datetime", "lpep_dropoff_datetime":"dropoff_datetime"})
df2 = df2.rename(columns={"tpep_pickup_datetime": "pickup_datetime", "tpep_dropoff_datetime":"dropoff_datetime"})
df1['cab'] = "green"
df2['cab'] = "yellow"

DatabaseError: Execution failed on sql 'SELECT * FROM green_cabs ORDER BY random() LIMIT 500000': no such table: green_cabs

In [4]:
# retain only columns that are in both datasets
cols_to_keep = df1.columns.intersection(df2.columns)
df1 = df1[cols_to_keep]
df2 = df2[cols_to_keep]
del cols_to_keep

# Combine into one data frame:
df = pd.concat([df1, df2], sort=False)
del df1
del df2

# Change format to datetime where necessary:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

df.head()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,congestion_surcharge,Source_file,cab
0,2.0,2019-07-25 00:25:38,2019-07-25 00:38:35,N,1.0,134,177,2.0,5.68,18.0,0.5,0.5,0.00,0.0,0.3,19.30,2.0,0.00,Data/green_tripdata_2019-07.csv,green
1,2.0,2019-10-20 13:02:09,2019-10-20 13:05:09,N,1.0,97,97,1.0,0.46,4.0,0.0,0.5,1.44,0.0,0.3,6.24,1.0,0.00,Data/green_tripdata_2019-10.csv,green
2,2.0,2019-10-27 16:08:51,2019-10-27 16:22:22,N,1.0,75,238,1.0,2.55,11.5,0.0,0.5,0.00,0.0,0.3,15.05,2.0,2.75,Data/green_tripdata_2019-10.csv,green
3,2.0,2019-04-15 17:48:52,2019-04-16 17:37:08,N,1.0,97,232,1.0,3.21,13.5,1.0,0.5,0.00,0.0,0.3,18.05,2.0,2.75,Data/green_tripdata_2019-04.csv,green
4,2.0,2019-01-22 15:12:56,2019-01-22 15:31:01,N,1.0,192,157,2.0,5.24,18.0,0.0,0.5,0.00,0.0,0.3,18.80,1.0,NaN,Data/green_tripdata_2019-01.csv,green


In [4]:
df.describe()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,congestion_surcharge
count,964559.000000,964559.000000,1000000.000000,1000000.000000,964559.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,964559.000000,892130.000000
mean,1.739934,1.121586,135.861190,145.358989,1.438791,3.121277,13.840889,0.811631,0.488645,1.590510,0.318130,0.283886,18.148505,1.361568,1.344370
std,0.449270,0.811134,74.157795,75.179726,1.103639,19.147255,12.777965,1.101522,0.082738,2.512427,1.668702,0.070867,14.840977,0.504181,1.268829
min,1.000000,1.000000,1.000000,1.000000,0.000000,-16183.310000,-530.000000,-7.000000,-0.500000,-21.220000,-18.500000,-0.300000,-530.800000,1.000000,-2.750000
25%,1.000000,1.000000,74.000000,75.000000,1.000000,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,9.800000,1.000000,0.000000
50%,2.000000,1.000000,137.000000,145.000000,1.000000,1.780000,10.000000,0.500000,0.500000,1.000000,0.000000,0.300000,13.800000,1.000000,2.500000
75%,2.000000,1.000000,196.000000,226.000000,1.000000,3.500000,16.000000,1.000000,0.500000,2.360000,0.000000,0.300000,20.800000,2.000000,2.500000
max,4.000000,99.000000,265.000000,265.000000,9.000000,135.530000,1957.000000,27.600000,3.550000,283.000000,618.130000,0.300000,1958.800000,5.000000,2.750000


### Remove outliers

In [5]:
df.quantile(q=np.array(range(0,11))*0.1)

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,congestion_surcharge
0.0,1.0,1.0,1.0,1.0,0.0,-40.25,-250.0,-4.50,-0.50,-24.79,-20.00,-0.30,-250.00,1.0,-2.75
0.1,1.0,1.0,41.0,42.0,1.0,0.61,5.0,0.00,0.50,0.00,0.00,0.30,7.30,1.0,0.00
0.2,1.0,1.0,66.0,69.0,1.0,0.90,6.0,0.00,0.50,0.00,0.00,0.30,9.30,1.0,0.00
0.3,2.0,1.0,79.0,90.0,1.0,1.13,7.0,0.00,0.50,0.00,0.00,0.30,10.56,1.0,0.00
0.4,2.0,1.0,100.0,129.0,1.0,1.41,8.5,0.00,0.50,0.00,0.00,0.30,12.15,1.0,0.00
0.5,2.0,1.0,137.0,145.0,1.0,1.77,10.0,0.50,0.50,1.00,0.00,0.30,13.80,1.0,2.50
0.6,2.0,1.0,161.0,164.0,1.0,2.24,11.5,0.50,0.50,1.65,0.00,0.30,15.95,1.0,2.50
0.7,2.0,1.0,170.0,196.0,1.0,2.96,14.5,1.00,0.50,2.06,0.00,0.30,18.80,2.0,2.50
0.8,2.0,1.0,230.0,233.0,2.0,4.25,19.0,1.00,0.50,2.76,0.00,0.30,23.76,2.0,2.50
0.9,2.0,1.0,239.0,239.0,2.0,7.43,28.0,2.75,0.50,3.96,0.00,0.30,34.04,2.0,2.50


Based on the above quantiles, the following filters should be applied:
- trip_distance > 0
- fare_amount > 0
- extra >= 0
- mta_tax >= 0
- tip_amount >= 0
- tolls_amount >= 0
- improvement_surcharge >= 0
- total_amount > 0
- congestion_surcharge >= 0

The upper limits should also be capped at the 99.9th percentile for:
- trip_distance
- fare_amount
- trip_time

Additionally, we only want to look at credit transactions because cash transactions are less likely to have a tip registered:
- payment_type == 1

In [6]:
indices = (df['trip_distance'] > 0) &\
    (df['fare_amount'] > 0) &\
    (df['extra'] >= 0) &\
    (df['mta_tax'] >= 0) &\
    (df['tip_amount'] >= 0) &\
    (df['tolls_amount'] >= 0) &\
    (df['improvement_surcharge'] >= 0) &\
    (df['total_amount'] > 0) &\
    (df['congestion_surcharge'] >= 0) &\
    (df['payment_type'] == 1) &\
    (df['trip_distance'] <= df['trip_distance'].quantile(.999)) &\
    (df['fare_amount'] <= df['fare_amount'].quantile(.999))

df_cleaned = df[indices]
del indices
df_cleaned.shape

(568322, 20)

In [7]:
df.shape

(1000000, 20)

### Check for NAs

In [7]:
df_cleaned.isna().sum()

VendorID                 0
pickup_datetime          0
dropoff_datetime         0
store_and_fwd_flag       0
RatecodeID               0
PULocationID             0
DOLocationID             0
passenger_count          0
trip_distance            0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
payment_type             0
congestion_surcharge     0
Source_file              0
cab                      0
dtype: int64

In [8]:
df = df_cleaned
del df_cleaned

## Feature engineering

### Add in borough information

In [9]:
# Store Pickup location borough names and merge to main data frame:
df_location = pd.read_csv("Data/taxi_zones.csv")
df_location['DOLocationID'] = df_location['LocationID']
df_location = df_location.rename(columns = {'LocationID':'PULocationID'})

# Add borough and Zone name:
df = df.merge(df_location[['PULocationID', 'borough', 'zone']], on = "PULocationID", how = 'left')

### Add weather information

In [30]:
# Store Weather informaiton and merge to main data frame

df_weather = pd.read_csv("Data/CP.weather.df.csv")

# convert Date to datetime
df_weather['Date'] = pd.to_datetime(df_weather['Date'])
df_weather = df_weather.rename(columns={"Date":"date"})

df['date'] = pd.to_datetime(df['pickup_datetime'].dt.date)

# merge with pickup date
df = df.merge(df_weather, how= 'left')


In [31]:
# Summary stats and structure of data:
df.head()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,zone,Date,Precipitation,Snow.depth,Snowfall,Max.temp,Min.temp,Avg.wind.speed,Gust.speed,date
0,2.0,2019-10-20 13:02:09,2019-10-20 13:05:09,N,1.0,97,97,1.0,0.46,4.0,...,Fort Greene,2019-10-20,0.54,0.0,0.0,59,49,4.5,14.1,2019-10-20
1,2.0,2019-10-11 23:58:15,2019-10-12 00:08:22,N,1.0,33,65,1.0,1.13,8.0,...,Brooklyn Heights,2019-10-11,0.00,0.0,0.0,64,55,9.2,19.9,2019-10-11
2,1.0,2019-05-20 12:04:37,2019-05-20 12:45:52,N,1.0,72,228,1.0,6.70,29.0,...,East Flatbush/Remsen Village,2019-05-20,0.00,0.0,0.0,85,66,5.4,18.1,2019-05-20
3,2.0,2019-07-06 19:10:47,2019-07-06 19:14:04,N,1.0,7,179,1.0,0.55,4.5,...,Astoria,2019-07-06,0.00,0.0,0.0,90,76,3.4,10.1,2019-07-06
4,2.0,2019-05-01 20:16:04,2019-05-01 20:20:07,N,1.0,166,151,1.0,0.79,5.0,...,Morningside Heights,2019-05-01,0.00,0.0,0.0,56,50,2.5,8.9,2019-05-01


### Add the following variables

- trip time
- average speed
- day of week
- holiday dummy code

In [12]:
# Include trip time:
df['trip_time'] = (df['dropoff_datetime'] - df['pickup_datetime']).astype('timedelta64[m]')

# Create average speed
df['avg_speed'] = df["trip_distance"]/(df["trip_time"]/60)

# Create day of pickup
df['day'] = df['pickup_datetime'].dt.day_name()

In [13]:
# Create holiday
cal = calendar()
holidays = cal.holidays(start=df['date'].min(), end=df['date'].max(), return_name=True)
holidays = holidays.reset_index(name='holiday').rename(columns={'index':'date'})
holidays['date'] = pd.to_datetime(holidays['date']).dt.date
df = pd.merge(df, holidays, on = 'date', how='left')
# create dummys for holiday 
df = pd.get_dummies(df,dummy_na= True, columns= ['holiday'])

# add nye
df['holiday_NYE'] = np.where((pd.to_datetime(df['date']).dt.month == 12) & \
                     (pd.to_datetime(df['date']).dt.day == 31), 1, 0)

# hour of day variable
hour = pd.get_dummies(df['pickup_datetime'].dt.hour, dummy_na= True)
hour.columns = ['hour_' + str(col) for col in hour.columns]
df = pd.concat([df, hour], axis=1)

# clean up column names
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.0$', '')

In [14]:
df.head()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_nan
0,2.0,2019-12-21 22:46:57,2019-12-21 23:09:22,N,1.0,7,131,2.0,10.87,31.0,...,0,0,0,0,0,0,0,1,0,0
1,2.0,2019-05-20 15:30:56,2019-05-20 15:47:09,N,1.0,196,130,1.0,3.16,13.5,...,1,0,0,0,0,0,0,0,0,0
2,2.0,2019-07-17 07:12:04,2019-07-17 07:19:42,N,1.0,33,231,1.0,2.21,9.0,...,0,0,0,0,0,0,0,0,0,0
3,2.0,2019-03-16 20:36:02,2019-03-16 20:47:53,N,1.0,95,134,1.0,1.80,9.5,...,0,0,0,0,0,1,0,0,0,0
4,2.0,2019-03-05 20:36:26,2019-03-05 20:43:02,N,1.0,181,40,1.0,1.38,7.0,...,0,0,0,0,0,1,0,0,0,0


## Final dataset

In [15]:
df.columns

Index(['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag',
       'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count',
       'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type',
       'congestion_surcharge', 'Source_file', 'cab', 'Borough', 'Zone',
       'trip_time', 'avg_speed', 'day', 'date', 'holiday_Christmas',
       'holiday_Columbus_Day', 'holiday_Dr._Martin_Luther_King_Jr.',
       'holiday_July_4th', 'holiday_Labor_Day', 'holiday_MemorialDay',
       'holiday_New_Years_Day', 'holiday_Presidents_Day',
       'holiday_Thanksgiving', 'holiday_Veterans_Day', 'holiday_nan',
       'holiday_NYE', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
       'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
       'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 

In [16]:
df.describe()

,VendorID,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_nan
count,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,...,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.000000,566501.0
mean,1.728242,1.101474,141.216916,150.148070,1.450795,3.262967,13.880663,0.849641,0.496990,2.595918,...,0.055931,0.056648,0.063802,0.069204,0.064448,0.056695,0.054725,0.049642,0.039650,0.0
std,0.454790,0.581309,73.319469,74.161083,1.120076,3.835556,11.300854,1.113863,0.038961,2.587966,...,0.229789,0.231168,0.244400,0.253801,0.245550,0.231260,0.227444,0.217204,0.195137,0.0
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,1.000000,1.000000,75.000000,85.000000,1.000000,1.100000,7.000000,0.000000,0.500000,1.160000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,2.000000,1.000000,142.000000,152.000000,1.000000,1.880000,10.000000,0.500000,0.500000,2.060000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,2.000000,1.000000,220.000000,230.000000,1.000000,3.600000,16.000000,1.000000,0.500000,3.160000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,4.000000,5.000000,265.000000,265.000000,8.000000,28.110000,91.000000,17.500000,3.550000,266.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [1]:
df

NameError: name 'df' is not defined

### Split into train and test then write to database

In [18]:
train_df = df.sample(frac=0.8, random_state=44)
test_df = df.drop(train_df.index)

In [19]:
train_df.to_sql("train_yellow_green", con=con, if_exists='replace')
test_df.to_sql("test_yellow_green", con=con, if_exists='replace')

Ensure tables were written by returning all tables in database

In [20]:
cur = con.cursor()
cur.execute('SELECT name from sqlite_master where type= "table"')
cur.fetchall()

[('yellow_cabs',),
 ('green_cabs',),
 ('train_yellow_green',),
 ('test_yellow_green',)]